In [ ]:
#' ' means CPU whereas '/device:G:0' means GPU
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}!kill -9 -1MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [ ]:
import datetime
from google.colab import drive
from time import time
from datetime import datetime
from datetime import timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filtered_works = pd.read_csv("/content/drive/MyDrive/NREGAproject/newestfilteredworks.csv", index_col=0, parse_dates=True)
# filtered_works = filtered_works.drop(['workcode', 'work_name', 'sno.', '_merge', 'index', 'work_status', 'bhu_creationtime', 'bhu_serial_no', 'bhu_collection_sno', 'sanction_amount_in(lakh.)'
# ,'total_amount_paid_since_inception_in(lakh.)', 'master_work_category_name', 'total_mandays', 'no._of_units', 'is_secure', 'is_convergence', 'bhu_accuracy', 'bhu_observername'
# ,'bhu_path1', 'bhu_path2', 'agency_name'], axis=1)
filtered_works['district_name'] = filtered_works['district_name'].str.lower()
filtered_works['block_name'] = filtered_works['block_name'].str.lower()
filtered_works['map_gpname'] = filtered_works['map_gpname'].str.lower()
filtered_works['work_physically/_completed_date'] = pd.to_datetime(filtered_works['work_physically/_completed_date'])
filtered_works['work_started_date'] = pd.to_datetime(filtered_works['work_started_date'])
listofgps= filtered_works['map_gpname'].unique()

<ipython-input-4-e313472d227a>:8: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  filtered_works['work_physically/_completed_date'] = pd.to_datetime(filtered_works['work_physically/_completed_date'])
<ipython-input-4-e313472d227a>:9: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  filtered_works['work_started_date'] = pd.to_datetime(filtered_works['work_started_date'])


In [ ]:
listofgps

array(['abludu', 'handiganala', nan, 'anur', 'baktharahalli',
       'bashetnahalli', 'thimmanayakanahalli', 'chimangala',
       'devaramaulur', 'dibburahalli', 'talakayalabetta',
       'doddathekalahalli', 'palicherlu', 'ganjigunta', 'kundalagurki',
       'hosahudya', 'melur', 'jangamakote', 'hospate', 'kothanur',
       'y.hunasenahalli', 'kumbiganahalli', 'malamachanahalli', 'mallur',
       'nagamangala', 's.devaganahalli', 'sadali', 'thammanahalli',
       'thimmasandra', 'venkatapura', 'anoor', 'batlahallai',
       'bhoomishettihalli', 'mindigal', 'burudagunte', 'chinnasandra',
       'enigadale', 'hirekattigenahalli', 'irgampalli', 'raguttahalli',
       'kadadanamari', 'kaiwara', 'kattariguppe', 'korlaparthi',
       'kencharlahalli', 'kotagal', 'doddaganjur', 'munaganahalli',
       'nandiganahalli', 'santhekallahalli', 'upperpete', 'shettihalli',
       'talagavara', 'ulavadi', 'konapalli', 'yagavakote', 'billur',
       'chakavelu', 'puligallu', 'racheravu', 'nallagutlap

In [ ]:
len(listofgps)

238

In [ ]:
def extractdates(gp):
  gp_lower = gp.lower()
  assetinfo1 = filtered_works[filtered_works['map_gpname'] == gp_lower] #139 rows
  max1 = assetinfo1['work_physically/_completed_date'].max()
  maxdate1 = max1.strftime('%Y-%m-%d')
  maxyr1 = str(max1.year)
  min1 = assetinfo1['work_started_date'].min()
  mindate1 = min1.strftime('%Y-%m-%d')
  minyr1 = str(min1.year)
  month1 = str(min1.month)
  # print(month1)
  # print(minyr1, maxyr1)
  arrmin = mindate1.split('-')
  mindate1 = arrmin[0] + '-' + arrmin[1] + '-01'
  arrmax = maxdate1.split('-')
  maxdate1 = arrmax[0] + '-' + arrmax[1] + '-01'
  # print(mindate1, maxdate1)#dates of availability of assets

  return [minyr1, maxyr1, mindate1, maxdate1]

In [ ]:
  ndvi = pd.read_csv('/content/drive/MyDrive/NREGAproject/daily-ndvi.csv')
  rainfall = pd.read_csv('/content/drive/MyDrive/NREGAproject/cds-precipitation-radiation.csv')
  rainfall = rainfall.drop(['surface_net_solar_radiation', '.geo', 'system:index'], axis=1)
  ndvi['date'] = pd.to_datetime(ndvi['date'])

In [ ]:
def preprocessing(gp, rainfall):
  # ndvi = pd.read_csv('/content/drive/MyDrive/NREGAproject/daily-ndvi.csv')
  # rainfall = pd.read_csv('/content/drive/MyDrive/NREGAproject/cds-precipitation-radiation.csv')
  # rainfall.drop(['surface_net_solar_radiation', '.geo', 'system:index'], axis=1)
  global ndvi
  rainfall1 = rainfall.pivot_table(values='total_precipitation', columns='gp', index='date')
  rainfall1.reset_index(inplace= True)
  rainfall1 = rainfall1.rename_axis(None, axis=1)
  rainfall1['date'] = pd.to_datetime(rainfall1['date'], format="%Y-%m-%d").dt.date

  if gp in ndvi.columns:
      ndvi_abludu = ndvi.loc[:, ['date', gp]]
      ndvi_abludu.rename(columns = {gp :'daily_NDVI'}, inplace = True)
      rainfall_abludu = rainfall1.loc[:, ['date', gp]]
      rainfall_abludu.rename(columns = {gp :'daily_rainfall'}, inplace = True)
      mylist = extractdates(gp)

      ndvi_abludu['date'] = pd.to_datetime(ndvi_abludu['date'])
      ndvi_albudu = ndvi_abludu.resample('1M', on='date').max()
      rainfall_abludu['date'] = pd.to_datetime(rainfall_abludu['date'])
      rainfall_abludu = rainfall_abludu.resample('1M', on='date').sum()
      merged = pd.merge(rainfall_abludu, ndvi_abludu, how='outer', on='date').dropna()
      merged['date'] = pd.to_datetime(merged['date'])
      merged.set_index('date', inplace=True)
      merged.reset_index(inplace = True)
      ndvi_before =merged[merged.date.dt.year < int(mylist[0])]
      ndvi_after =merged[merged.date.dt.year >= int(mylist[0])]

      return [ndvi_before, ndvi_after]

In [ ]:
def graphdataset(dataset, gp):
  f = plt.figure()
  f.set_figwidth(15)
  f.set_figheight(4)

  ax1 = plt.subplot()
  l1, = ax1.plot(dataset['daily_NDVI'], color='green')
  ax2 = ax1.twinx()
  l2, = ax2.plot(dataset['daily_rainfall'], color='blue')
  ax1.set_xlabel('Years')
  ax1.set_ylabel('daily NDVI')
  ax2.set_ylabel('daily rainfall (m)')

  plt.legend([l1, l2], ["daily NDVI over the years in " + gp, " daily rainfall over the years"], bbox_to_anchor =(0.65, 1.25))
  plt.show()

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

In [ ]:
gp = 'Hospate'
mydatasets = preprocessing(gp, rainfall)
mydatasets[0].head()

,date,daily_rainfall,daily_NDVI
0,2000-02-29,0.055180,0.389086
1,2000-03-31,0.001205,0.378369
2,2000-04-30,0.037530,0.326623
3,2000-05-31,0.119797,0.025232
4,2000-06-30,0.101085,0.001155


In [ ]:
def grangerscausality(databefore, dataafter):
  try:
    diction = grangercausalitytests(databefore[['daily_NDVI', 'daily_rainfall']], [6]) #the predicted variable is put first, and then the predictor variable
    mydict = grangercausalitytests(dataafter[['daily_NDVI', 'daily_rainfall']], [6])
    num = diction[6][0]['ssr_ftest'][1]
    num2 = mydict[6][0]['ssr_ftest'][1]
  except ValueError as e:
    print(f"Error: {e}")
    # Handle the error gracefully, e.g., set num and num2 to some default values or return None
    num, num2 = None, None

  return [num, num2]

In [ ]:
df = pd.DataFrame(columns=['gpname','pbefore','pafter'])

for gp in listofgps:
  if isinstance(gp, str):
    gp = gp.capitalize()
    mydatasets = preprocessing(gp, rainfall)
    if mydatasets is None:
      continue
    else:
      mypvals = grangerscausality(mydatasets[0], mydatasets[1])
      df.loc[len(df.index)] = [gp, mypvals[0], mypvals[1]]


Granger Causality
number of lags (no zero) 6
ssr based F test:         F=3.3458  , p=0.0056  , df_denom=76, df_num=6
ssr based chi2 test:   chi2=23.5086 , p=0.0006  , df=6
likelihood ratio test: chi2=20.8610 , p=0.0019  , df=6
parameter F test:         F=3.3458  , p=0.0056  , df_denom=76, df_num=6

Granger Causality
number of lags (no zero) 6
ssr based F test:         F=3.4612  , p=0.0031  , df_denom=154, df_num=6
ssr based chi2 test:   chi2=22.5206 , p=0.0010  , df=6
likelihood ratio test: chi2=21.1261 , p=0.0017  , df=6
parameter F test:         F=3.4612  , p=0.0031  , df_denom=154, df_num=6

Granger Causality
number of lags (no zero) 6
ssr based F test:         F=3.2100  , p=0.0073  , df_denom=76, df_num=6
ssr based chi2 test:   chi2=22.5546 , p=0.0010  , df=6
likelihood ratio test: chi2=20.1031 , p=0.0027  , df=6
parameter F test:         F=3.2100  , p=0.0073  , df_denom=76, df_num=6

Granger Causality
number of lags (no zero) 6
ssr based F test:         F=3.2678  , p=0.0047  , df

In [ ]:
df.tail()

,gpname,pbefore,pafter
220,Chaldiganahalli,0.000104,0.616711
221,Abbenahalli,0.000004,0.850147
222,Soolikunte,0.000343,0.576753
223,Mavahalli,NaN,NaN
224,Madivala,0.000004,0.397166


In [ ]:
df[df.pbefore <= df.pafter].count()

In [ ]:
df[df.pbefore < df.pafter].count()

,gpname,pbefore,pafter


In [ ]:
# Assuming df is your DataFrame
df.to_csv('/content/drive/MyDrive/NREGAproject/grangersmodularizedfinal.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/NREGAproject/grangersmodularizedfinal.csv', index_col=0, parse_dates=True)

In [ ]:
df[df.pbefore <= df.pafter].count()

pbefore    68
pafter     68
dtype: int64

## Checking stationarity of these series

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
df2 = pd.DataFrame(columns=['gpname','aftndvi','adfrainfall'])

In [ ]:
rainfall.head()

,date,gp,total_precipitation
0,2000-01-01T23:00:00,Chelur,0.000454
1,2000-01-01T23:00:00,Kothakote,0.000311
2,2000-01-01T23:00:00,Mittemari,0.000265
3,2000-01-01T23:00:00,Nandi,0.000077
4,2000-01-01T23:00:00,Doddaganjur,0.000461


In [ ]:
def adf_test(timeseries):
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    print(dfoutput['p-value'])
    return(dfoutput["p-value"])

In [ ]:
for gp in listofgps:
  gp = gp.capitalize()
  mydatasets2 = preprocessing(gp, rainfall)
  if mydatasets is None:
    continue
  else:
    mergedmydatasets = mydatasets2[0].append(mydatasets[1], ignore_index=True)
    adfndvi = adf_test(mergedmydatasets['daily_NDVI'])
    adfrainfall = adf_test(mergedmydatasets['daily_rainfall'])
    df2.loc[len(df.index)] = [gp, adfndvi, adfrainfall]

KeyboardInterrupt: ignored